In [2]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 构建Inception

In [5]:
class Inception(keras.layers.Layer):
    def __init__(self, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj, **kwargs):
        super().__init__(**kwargs)
        self.branch1 = keras.layers.Conv2D(ch1x1, kernel_size=1, activation='relu')
        
        self.branch2 = keras.Sequential([
            keras.layers.Conv2D(ch3x3red, kernel_size=1, activation='relu'),
            keras.layers.Conv2D(ch3x3, kernel_size=3, padding='SAME', activation='relu')
        ])
        
        self.branch3 = keras.Sequential([
            keras.layers.Conv2D(ch5x5red, kernel_size=1, activation='relu'),
            keras.layers.Conv2D(ch5x5, kernel_size=5, padding='SAME', activation='relu')
        ])
        
        self.branch4 = keras.Sequential([
            keras.layers.MaxPool2D(pool_size=3, strides=1, padding='SAME'),
            keras.layers.Conv2D(pool_proj, kernel_size=1, activation='relu')
        ])
        
    def call(self, inputs, **kwargs):
        branch1 = self.branch1(inputs)
        branch2 = self.branch2(inputs)
        branch3 = self.branch3(inputs)
        branch4 = self.branch4(inputs)
        outputs = keras.layers.concatenate([branch1, branch2, branch3, branch4])
        return outputs

# 定义辅助输出结构

In [6]:
class InceptionAux(keras.layers.Layer):
    def __init__(self, num_classes, **kwargs):
        super().__init__(**kwargs)
        self.average_pool = keras.layers.AvgPool2D(pool_size=5, strides=3)
        self.conv = keras.layers.Conv2D(128, kernel_size=1, activation='relu')
        
        self.fc1 = keras.layers.Dense(1024, activation='relu')
        self.fc2 = keras.layers.Dense(num_classes)
        self.softmax = keras.layers.Softmax()
        
    def call(self, inputs, **kwargs):
        x = self.average_pool(inputs)
        x = self.conv(x)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dropout(rate=0.5)(x)
        x = self.fc1(x)
        x = keras.layers.Dropout(rate=0.5)(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# 定义网络

In [7]:
def GoogLeNet(im_height=224, im_width=224, class_num=1000, aux_logits=False):
    input_image = keras.layers.Input(shape=(im_height, im_width, 3), dtype='float32')
    x = keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='SAME', activation='relu')(input_image)
    # 注意MaxPool2D, padding='SAME', 224/2=112, padding='VALID', (224 -(3 -1 )) / 2 = 111, same向上取整. 
    x = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='SAME')(x)
    x = keras.layers.Conv2D(64, kernel_size=1, strides=1, padding='SAME', activation='relu')(x)
    x = keras.layers.Conv2D(192, kernel_size=3, strides=1, padding='SAME', activation='relu')(x)
    x = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='SAME')(x)
    
    x = Inception(64, 96, 128, 16, 32, 32, name='inception_3a')(x)
    x = Inception(128, 128, 192, 32, 96, 64, name='inception_3b')(x)
    
    x = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='SAME')(x)
    x = Inception(192, 96, 208, 16, 48, 64, name='inception_4a')(x)
    
    if aux_logits:
        aux1 = InceptionAux(class_num, name='aux_1')(x)
        
    x = Inception(160, 112, 224, 24, 64, 64, name='inception_4b')(x)
    x = Inception(128, 128, 256, 24, 64, 64, name='inception_4c')(x)
    x = Inception(112, 144, 288, 32, 64, 64, name='inception_4d')(x)
    
    if aux_logits:
        aux2 = InceptionAux(class_num, name='aux_2')(x)
        
    x = Inception(256, 160, 320, 32, 128, 128, name='inception_4e')(x)
    
    x = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='SAME')(x)
    
    x = Inception(256, 160, 320, 32, 128, 128, name='inception_5a')(x)
    x = Inception(384, 192, 384, 48, 128, 128, name='inception_5b')(x)
    
    x = keras.layers.AvgPool2D(pool_size=7, strides=1)(x)
    
    x = keras.layers.Flatten()(x)
    
    x = keras.layers.Dropout(rate=0.4)(x)
    x = keras.layers.Dense(class_num)(x)
    aux3 = keras.layers.Softmax(name='aux_3')(x)
    
    if aux_logits:
        aux = aux1 * 0.2 + aux2 * 0.3 + aux3 * 0.5
        model = keras.models.Model(inputs=input_image, outputs=aux)
    else:
        model = keras.models.Model(inputs=input_image, outputs=aux3)
    return model

# 数据预处理和导入

In [8]:
train_dir = r'C:/Users/Administrator/Desktop/data/archive/training/training/'
valid_dir =r'C:/Users/Administrator/Desktop/data/archive/validation/validation/'

# 图片数据生成器
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

height = 224
width = 224
channels = 3
batch_size = 32
num_classes = 10

train_generator = train_datagen.flow_from_directory(train_dir,
                                 target_size = (height, width),
                                 batch_size = batch_size,
                                 shuffle = True,
                                 seed = 7,
                                 class_mode = 'categorical')

valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1. / 255
)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                 target_size = (height, width),
                                 batch_size = batch_size,
                                 shuffle = True,
                                 seed = 7,
                                 class_mode = 'categorical')
print(train_generator.samples)
print(valid_generator.samples)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098
272


# 构建网络

In [9]:
googlenet = GoogLeNet(class_num=10,aux_logits=True)

In [10]:
googlenet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                              

In [11]:
googlenet.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['acc'])

# 训练网络

In [12]:
history = googlenet.fit(train_generator,
                   steps_per_epoch=train_generator.samples // batch_size,
                   epochs=10,
                   validation_data=valid_generator,
                   validation_steps = valid_generator.samples // batch_size
                   )

Epoch 1/10
34/34 [==============================] - 87s 2s/step - loss: 2.3083 - acc: 0.1013 - val_loss: 2.3018 - val_acc: 0.1133
Epoch 2/10
34/34 [==============================] - 82s 2s/step - loss: 2.3031 - acc: 0.1098 - val_loss: 2.2887 - val_acc: 0.1133
Epoch 3/10
34/34 [==============================] - 82s 2s/step - loss: 2.3044 - acc: 0.0919 - val_loss: 2.3021 - val_acc: 0.1055
Epoch 4/10
34/34 [==============================] - 82s 2s/step - loss: 2.3010 - acc: 0.0994 - val_loss: 2.2884 - val_acc: 0.1016
Epoch 5/10
34/34 [==============================] - 83s 2s/step - loss: 2.2691 - acc: 0.1839 - val_loss: 2.2467 - val_acc: 0.1406
Epoch 6/10
34/34 [==============================] - 82s 2s/step - loss: 2.1671 - acc: 0.1886 - val_loss: 2.1048 - val_acc: 0.2031
Epoch 7/10
34/34 [==============================] - 83s 2s/step - loss: 2.1132 - acc: 0.2083 - val_loss: 2.0490 - val_acc: 0.2578
Epoch 8/10
34/34 [==============================] - 85s 2s/step - loss: 2.0650 - acc: 0.21

# 预测

In [16]:
history.history

{'loss': [2.3082504272460938,
  2.3031396865844727,
  2.3044490814208984,
  2.301003932952881,
  2.2691454887390137,
  2.167141914367676,
  2.113212823867798,
  2.065011978149414,
  2.0462825298309326,
  2.0393829345703125],
 'acc': [0.10131332278251648,
  0.10975609719753265,
  0.09193246066570282,
  0.09943714737892151,
  0.18386492133140564,
  0.18855534493923187,
  0.20825515687465668,
  0.21575984358787537,
  0.2138836830854416,
  0.24296435713768005],
 'val_loss': [2.301816940307617,
  2.288682460784912,
  2.3020541667938232,
  2.2883920669555664,
  2.2466745376586914,
  2.1047840118408203,
  2.04896879196167,
  2.0972814559936523,
  2.1115286350250244,
  2.0270183086395264],
 'val_acc': [0.11328125,
  0.11328125,
  0.10546875,
  0.1015625,
  0.140625,
  0.203125,
  0.2578125,
  0.21875,
  0.21484375,
  0.2265625]}